In [65]:
import pandas as pd
import datetime
import numpy as np
import pandas as pd
import datetime

In [66]:
dfAtual = pd.read_csv('Base_de_dados/dados_limpos.csv', sep=';')
dfAtual['dataCaso'] = pd.to_datetime(dfAtual['dataCaso'])
# data['dataCaso'] = data['dataCaso'].dt.date

A:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [67]:
filtroDt = (dfAtual.dataCaso >= datetime.datetime(2020, 3, 1)) & (dfAtual.dataCaso <= datetime.datetime(2021, 10, 7))
df1 = dfAtual[filtroDt]

In [68]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1370752 entries, 0 to 1372750
Data columns (total 11 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   Unnamed: 0             1370752 non-null  int64         
 1   identificadorCaso      1370752 non-null  object        
 2   profissionalSaudeEsus  603398 non-null   object        
 3   bairroCaso             1370752 non-null  object        
 4   sexoCaso               1370752 non-null  object        
 5   idadeCaso              1369015 non-null  float64       
 6   faixaEtaria            1370752 non-null  object        
 7   resultadoFinalExame    1370752 non-null  object        
 8   obitoConfirmado        1370752 non-null  object        
 9   dataCaso               1370752 non-null  datetime64[ns]
 10  profissoes             36702 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(8)
memory usage: 125.5+ MB


In [69]:
df1.faixaEtaria.value_counts().index.to_list()

['35 a 39 anos',
 '30 a 34 anos',
 '40 a 44 anos',
 '25 a 29 anos',
 '20 a 24 anos',
 '45 a 49 anos',
 '50 a 54 anos',
 '55 a 59 anos',
 '60 a 64 anos',
 '15 a 19 anos',
 '65 a 69 anos',
 '00 a 04 anos',
 '70 a 74 anos',
 '80 ou mais',
 '10 a 14 anos',
 '05 a 09 anos',
 '75 a 79 anos',
 'Sem Informacao']

# COISAS QUE QUERO TRANSFORMAR EM NUMERICA:
- Numero de obitos (FOI)
- Numero de casos positivos (FOI)
- Media de idade por dia (FOI)
- Obitos e casos por sexo (FOI)
- Obitos e casos por raça por faixa de idade

# ==============================================================================

# Refatorando faixa etaria

In [70]:
df1.faixaEtaria.replace('00 a 04 anos', '00 a 19 anos', inplace=True)
df1.faixaEtaria.replace('05 a 09 anos', '00 a 19 anos', inplace=True)
df1.faixaEtaria.replace('10 a 14 anos', '00 a 19 anos', inplace=True)
df1.faixaEtaria.replace('15 a 19 anos', '00 a 19 anos', inplace=True)
df1.faixaEtaria.replace('20 a 24 anos', '20 a 39 anos', inplace=True)
df1.faixaEtaria.replace('25 a 29 anos', '20 a 39 anos', inplace=True)
df1.faixaEtaria.replace('30 a 34 anos', '20 a 39 anos', inplace=True)
df1.faixaEtaria.replace('35 a 39 anos', '20 a 39 anos', inplace=True)
df1.faixaEtaria.replace('40 a 44 anos', '40 a 59 anos', inplace=True)
df1.faixaEtaria.replace('45 a 49 anos', '40 a 59 anos', inplace=True)
df1.faixaEtaria.replace('50 a 54 anos', '40 a 59 anos', inplace=True)
df1.faixaEtaria.replace('55 a 59 anos', '40 a 59 anos', inplace=True)
df1.faixaEtaria.replace('60 a 64 anos', '60 a 79 anos', inplace=True)
df1.faixaEtaria.replace('65 a 69 anos', '60 a 79 anos', inplace=True)
df1.faixaEtaria.replace('70 a 74 anos', '60 a 79 anos', inplace=True)
df1.faixaEtaria.replace('75 a 79 anos', '60 a 79 anos', inplace=True)

A:\anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


# ==============================================================================

# Numero de casos confirmados

In [71]:
filtroConfirmado = (df1.resultadoFinalExame == 'Positivo')
dfConfirmado = df1[filtroConfirmado]

grouped = dfConfirmado.groupby(['dataCaso', 'bairroCaso', 'faixaEtaria'])['resultadoFinalExame'].count().reset_index().sort_values('dataCaso')

num_confirmado = {'data': grouped.dataCaso.to_list(), 'bairro': grouped.bairroCaso.to_list(), 'faixa Etaria': grouped.faixaEtaria.to_list(), 'casos confirmados': grouped.resultadoFinalExame.to_list()}
num_confirmado = pd.DataFrame(num_confirmado)

In [72]:
num_confirmado.set_index(['data', 'bairro', 'faixa Etaria'], inplace=True)
num_confirmado.head()

casos confirmados
data       bairro          faixa Etaria                   
2020-03-01 AEROPORTO       60 a 79 anos                  1
           JACARECANGA     20 a 39 anos                  2
                           40 a 59 anos                  2
           JARDIM AMERICA  20 a 39 anos                  1
           JOSE DE ALENCAR 20 a 39 anos                  1

# ==============================================================================

# Numero de obitos

In [73]:
filtroObito = (df1.obitoConfirmado == 'Verdadeiro')
dfObito = df1[filtroObito]

grouped = dfObito.groupby(['dataCaso', 'bairroCaso', 'faixaEtaria'])['obitoConfirmado'].count().reset_index().sort_values('dataCaso')

num_obitos = {'data': grouped.dataCaso.to_list(), 'bairro': grouped.bairroCaso.to_list(), 'faixa Etaria': grouped.faixaEtaria.to_list(), 'obitos': grouped.obitoConfirmado.to_list()}
num_obitos = pd.DataFrame(num_obitos)

In [74]:
num_obitos.set_index(['data', 'bairro', 'faixa Etaria'], inplace=True)
num_obitos.head()

,,,obitos
data,bairro,faixa Etaria,
2020-03-01,Indeterminado,80 ou mais,1
2020-03-02,CAMBEBA,60 a 79 anos,1
2020-03-05,DIONISIO TORRES,20 a 39 anos,1
2020-03-09,CENTRO,60 a 79 anos,1
2020-03-10,CONJUNTO CEARA I,60 a 79 anos,1


# ==============================================================================

# Numeros segmentados pelo sexo

## Confirmados

In [75]:
grouped = dfConfirmado.groupby(['sexoCaso', 'bairroCaso', 'faixaEtaria', pd.Grouper(key='dataCaso', freq='D')])['resultadoFinalExame'].count().reset_index().sort_values('dataCaso')

In [76]:
grouped = pd.get_dummies(grouped, columns=['sexoCaso'])
grouped['confirmados_masculinos'] = np.where(grouped['sexoCaso_MASCULINO'] == 1, grouped['resultadoFinalExame'], 0)
grouped['confirmados_feminino'] = np.where(grouped['sexoCaso_FEMININO'] == 1, grouped['resultadoFinalExame'], 0)
grouped.drop(columns=['resultadoFinalExame', 'sexoCaso_FEMININO', 'sexoCaso_MASCULINO'], inplace=True)

In [77]:
aggregation_functions = {'confirmados_masculinos': 'sum', 'confirmados_feminino': 'sum'}
num_confirmado_sexo = grouped.groupby(['dataCaso', 'bairroCaso', 'faixaEtaria']).aggregate(aggregation_functions)

In [78]:
num_confirmado_sexo.head()

confirmados_masculinos  \
dataCaso   bairroCaso      faixaEtaria                            
2020-03-01 AEROPORTO       60 a 79 anos                       0   
           ALDEOTA         20 a 39 anos                       1   
                           40 a 59 anos                       0   
           ANTONIO BEZERRA 20 a 39 anos                       1   
                           60 a 79 anos                       1   

                                         confirmados_feminino  
dataCaso   bairroCaso      faixaEtaria                         
2020-03-01 AEROPORTO       60 a 79 anos                     1  
           ALDEOTA         20 a 39 anos                     0  
                           40 a 59 anos                     2  
           ANTONIO BEZERRA 20 a 39 anos                     0  
                           60 a 79 anos                     0

## Obitos

In [79]:
grouped = dfObito.groupby(['sexoCaso', 'bairroCaso', 'faixaEtaria', pd.Grouper(key='dataCaso', freq='D')])['obitoConfirmado'].count().reset_index().sort_values('dataCaso')

In [80]:
grouped = pd.get_dummies(grouped, columns=['sexoCaso'])
grouped['obitos_masculinos'] = np.where(grouped['sexoCaso_MASCULINO'] == 1, grouped['obitoConfirmado'], 0)
grouped['obitos_feminino'] = np.where(grouped['sexoCaso_FEMININO'] == 1, grouped['obitoConfirmado'], 0)
grouped.drop(columns=['obitoConfirmado', 'sexoCaso_FEMININO', 'sexoCaso_MASCULINO'], inplace=True)

In [81]:
aggregation_functions = {'obitos_masculinos': 'sum', 'obitos_feminino': 'sum'}
num_obitos_sexo = grouped.groupby(['dataCaso', 'bairroCaso', 'faixaEtaria']).aggregate(aggregation_functions)

In [82]:
num_obitos_sexo.head()

,,,obitos_masculinos,obitos_feminino
dataCaso,bairroCaso,faixaEtaria,,
2020-03-01,Indeterminado,80 ou mais,1,0
2020-03-02,CAMBEBA,60 a 79 anos,1,0
2020-03-05,DIONISIO TORRES,20 a 39 anos,1,0
2020-03-09,CENTRO,60 a 79 anos,1,0
2020-03-10,CONJUNTO CEARA I,60 a 79 anos,0,1


# ==============================================================================

# Juntando tudo

In [83]:
base = pd.concat([num_confirmado, num_obitos, med_idade_confirmados, num_confirmado_sexo, num_obitos_sexo], axis=1)
base.fillna(0, inplace=True)
base.head()

casos confirmados  obitos  \
2020-03-01 AEROPORTO       60 a 79 anos                1.0     0.0   
           ALDEOTA         20 a 39 anos                1.0     0.0   
                           40 a 59 anos                2.0     0.0   
           ANTONIO BEZERRA 20 a 39 anos                1.0     0.0   
                           60 a 79 anos                1.0     0.0   

                                         media de idade  \
2020-03-01 AEROPORTO       60 a 79 anos            74.0   
           ALDEOTA         20 a 39 anos            35.0   
                           40 a 59 anos            45.0   
           ANTONIO BEZERRA 20 a 39 anos            21.0   
                           60 a 79 anos            63.0   

                                         confirmados_masculinos  \
2020-03-01 AEROPORTO       60 a 79 anos                     0.0   
           ALDEOTA         20 a 39 anos                     1.0   
                           40 a 59 anos                     0.0   
           ANTONIO BEZERRA 20 a 39 anos                     1.0   
                           60 a 79 anos                     1.0   

                                         confirmados_feminino  \
2020-03-01 AEROPORTO       60 a 79 anos                   1.0   
           ALDEOTA         20 a 39 anos                   0.0   
                           40 a 59 anos                   2.0   
           ANTONIO BEZERRA 20 a 39 anos                   0.0   
                           60 a 79 anos                   0.0   

                                         obitos_masculinos  obitos_feminino  
2020-03-01 AEROPORTO       60 a 79 anos                0.0              0.0  
           ALDEOTA         20 a 39 anos                0.0              0.0  
                           40 a 59 anos                0.0              0.0  
           ANTONIO BEZERRA 20 a 39 anos                0.0              0.0  
                           60 a 79 anos                0.0              0.0

In [84]:
base.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 95503 entries, (Timestamp('2020-03-01 00:00:00'), 'AEROPORTO', '60 a 79 anos') to (Timestamp('2021-07-25 00:00:00'), 'Indeterminado', '40 a 59 anos')
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   casos confirmados       95503 non-null  float64
 1   obitos                  95503 non-null  float64
 2   media de idade          95503 non-null  float64
 3   confirmados_masculinos  95503 non-null  float64
 4   confirmados_feminino    95503 non-null  float64
 5   obitos_masculinos       95503 non-null  float64
 6   obitos_feminino         95503 non-null  float64
dtypes: float64(7)
memory usage: 5.5+ MB


In [85]:
base.to_csv('Base_de_dados.csv', sep=';')